In [0]:
%sql
CREATE OR REPLACE TEMP VIEW fiscal_cal AS
SELECT
    DAY_DT,
    DAY_OF_WK_NUM,
    WK_END_DT,
    CAST(FISC_YR_CD AS INT) AS FISC_YR_CD,
    CAST(CONCAT(FISC_YR_CD, LPAD(FISC_WK_OF_YR_NUM, 2, '0')) AS INT) AS fiscal_week_num,
    FISC_WK_OF_YR_NUM,
    CAST(CONCAT(FISC_YR_CD, LPAD(FISC_PRD_OF_YR_NUM, 2, '0')) AS INT) AS fiscal_period_num,
    FISC_PRD_OF_YR_NUM,
    CAST(CONCAT(FISC_YR_CD, LPAD(FISC_QTR_OF_YR_NUM, 2, '0')) AS INT) AS fiscal_qtr_num,
    FISC_QTR_OF_YR_NUM
FROM main.uc_aa_datamart.fiscal_calendar;


In [0]:
%sql
create or replace table `test-analytics`.custom_datamart.pickrate_perf_food_lion as 
(with emp_perf as (select *,
       -- case when trim(arg2_cd) = 'M' then 'Multi'
       case when trim(arg2_cd) in ('C','center') then 'CenterStore'
       when trim(arg2_cd) in ('F','frozen') then 'Frozen'
       -- when trim(arg2_cd) = 'R' then 'Repick'
       when trim(arg2_cd) in ('P','perimeter') then 'Perimeter'
       -- when trim(arg2_cd) = 'Y' then 'Unknown'
       else 'Multi'
       end as arg_desc
       from (select distinct * from merchandising.pos.emp_perf_fdln_sdm)),
emp_final as (select
       trim(ee_id) ee_id, 
       arg_desc, 
       date(dlv_dt) date, 
       trim(dc_id) dc_id_a,
       IFNULL(sum(cnt1_qy),0) as units_pick, 
       count(perf_id) picks,
       sum(date_diff(second, beg_dt_tm,end_dt_tm)) actv_time
from emp_perf where task_id = 'P'
group by ee_id, arg_desc, date(dlv_dt), trim(dc_id)),
actv as (select 
        trim(a.dc_id) dc_id, 
        trim(a.ee_id) ee_id, 
        date(a.actv_dt_tm) as date,
       sum(case when trim(actv_cd) in ('P', 'S') then arg_tx else 0 end) as pick_items,
       sum(case when trim(actv_cd) in ('O') then arg_tx else 0 end) as oos_items,
       sum(case when trim(actv_cd) in ('U') then arg_tx else 0 end) as sub_items,
       case when trim(arg2_cd) in ('C','center') then 'CenterStore'
       when trim(arg2_cd) in ('F','frozen') then 'Frozen'
       -- when trim(arg2_cd) = 'R' then 'Repick'
       when trim(arg2_cd) in ('P','perimeter') then 'Perimeter'
       -- when trim(arg2_cd) = 'Y' then 'Unknown'
       else 'Multi'
       end as arg_desc
from (select distinct * from merchandising.pos.amt_actv_trk_fdln_sdm) a left join (select distinct * from merchandising.pos.emp_perf_fdln_sdm) b on a.perf_id = b.perf_id AND date(a.actv_dt_tm) = date(b.dlv_dt) and a.dc_id = b.dc_id and a.ee_id = b.ee_id
group by a.ee_id, date(a.actv_dt_tm), trim(a.dc_id), 7),
last as (select a.*, b.pick_items, b.oos_items, b.sub_items, ab.stor_ttl_tx, 
    ab.st_cd, 
    ab.zip_cd, 
     ab.ahld_stor_nbr_cd,
     c.* from 
emp_final a left join actv b on a.dc_id_a = b.dc_id and a.arg_desc = b.arg_desc and a.ee_id = b.ee_id and a.`date` = b.`date`
 left outer JOIN managed_assets.location.cit_pup_ctl_sdm ab ON a.dc_id_a = ab.p3_stor_id
               left outer join fiscal_cal c on a.date = c.DAY_DT)
select date as dlv_dt,
       stor_ttl_tx,
       st_cd,
       zip_cd,
       ahld_stor_nbr_cd,
       dc_id_a as dc_id,
       ee_id,
       arg_desc as pick_desc_tx,
       sum(picks) as pick,
       sum(units_pick) as units_pck,
       sum(actv_time) as shft_secs,
       DAY_DT,
       DAY_OF_WK_NUM,
       WK_END_DT,
       FISC_YR_CD,
       fiscal_week_num,
       FISC_WK_OF_YR_NUM,
       fiscal_period_num,
       FISC_PRD_OF_YR_NUM,
       fiscal_qtr_num,
       FISC_QTR_OF_YR_NUM,
       sum(pick_items) tot_pick,
       sum(oos_items) oos_items,
       sum(sub_items) sub_items
       from last
group by all)

In [0]:
%sql
create or replace table `test-analytics`.custom_datamart.pickrate_perf_food_lion_all as (
with temp as (select date(actv_dt_tm) dt,
       ee_id,
       actv_cd,
       ord_id,
       dc_id,
       min(actv_dt_tm) beg, max(actv_dt_tm) end,
       sum(arg_tx) arg_tx,
       date_diff(second, min(actv_dt_tm), max(actv_dt_tm)) seconds,
       date_diff(minute, min(actv_dt_tm), max(actv_dt_tm)) hour from merchandising.pos.amt_actv_trk_fdln_sdm
       group by date(actv_dt_tm),
       actv_cd,
       ee_id,
       dc_id,
       ord_id),
last as (select dt,
              ee_id,
              dc_id,
              count(distinct ord_id) ords,
              IFNULL(sum(arg_tx) filter(where trim(actv_cd) = 'U'),0) as total_subs,
              IFNULL(sum(arg_tx) filter(where trim(actv_cd) in ('P', 'S')), 0) as total_picks,
              IFNULL(sum(arg_tx) filter(where trim(actv_cd) in ('O')),0) as total_oos,
              sum(seconds) seconds
              from temp
              group by dt,
              ee_id,
              dc_id)
select a.*,  b.stor_ttl_tx, 
    b.st_cd, 
    CAST(b.zip_cd as int), 
     CAST(b.ahld_stor_nbr_cd as int), c.* from last a left outer JOIN managed_assets.location.cit_pup_ctl_sdm b ON a.dc_id = b.p3_stor_id
               left outer join fiscal_cal c on a.dt = c.DAY_DT
)

In [0]:
%run "/Workspace/Users/nusxsa4@ad.peapod.com/Productivity Dashboard/Food Lion Spectrum(Prism)/FL Unata Picking USING RDS"